# Example of using TVB importers from within the REST client API.

Here, we upload two independent datatypes: a connectivity and a surface from ZIP formats.

Then, we upload a region mapping that depends on both connectivity and surface to exist in TVB storage.

In [1]:
import os
import time

import tvb_data
from keycloak import KeycloakOpenID
from tvb.core.entities.model.model_operation import STATUS_ERROR, STATUS_CANCELED, STATUS_FINISHED
from tvb.interfaces.rest.client.tvb_client import TVBClient

In [2]:
def compute_tvb_data_path(folder, filename):
    return os.path.join(os.path.dirname(tvb_data.__file__), folder, filename)

def monitor_operation(tvb_client, operation_gid):
    while True:
        status = tvb_client.get_operation_status(operation_gid)
        if status in [STATUS_FINISHED, STATUS_CANCELED, STATUS_ERROR]:
            break
        print("Current operation GID {} => STATUS {}".format(operation_gid, status))
        time.sleep(5)
    print("Operation {} has finished with status: {}".format(operation_gid, status))

In [3]:
tvb_client = TVBClient("http://192.168.123.93:9090")

keycloak_instance = KeycloakOpenID("https://keycloak.codemart.ro/auth/", "TVB", "tvb-tests")

tvb_client._update_token(keycloak_instance.token("tvb_user", "pass"))

In [4]:
# Requesting projects for the logged user
projects_of_user = tvb_client.get_project_list()
default_project = projects_of_user[0]
default_project_gid = default_project.gid
default_project.name

'Default_Project'

## Launch operations to import a Connectivity, a Surface and a RegionMapping

In [5]:
from tvb.adapters.uploaders.zip_connectivity_importer import ZIPConnectivityImporterModel, ZIPConnectivityImporter

# Importing a connectivity from ZIP
zip_connectivity_importer_model = ZIPConnectivityImporterModel()
zip_connectivity_importer_model.uploaded = compute_tvb_data_path('connectivity', 'connectivity_96.zip')
zip_connectivity_importer_model.normalization = 'region'
operation_gid = tvb_client.launch_operation(default_project_gid, ZIPConnectivityImporter,
                                                         zip_connectivity_importer_model)

monitor_operation(tvb_client, operation_gid)

# Get the result of connectivity import
connectivity_dto = tvb_client.get_operation_results(operation_gid)[0]
connectivity_gid = connectivity_dto.gid

connectivity_gid

Current operation GID a7899256-7b33-11ea-b3cb-eb2a08972b7e => STATUS 4-PENDING
Operation a7899256-7b33-11ea-b3cb-eb2a08972b7e has finished with status: 5-FINISHED


'f247b79e6b1d449da6599e235494c32e'

In [6]:
from tvb.adapters.uploaders.zip_surface_importer import ZIPSurfaceImporterModel, ZIPSurfaceImporter
from tvb.datatypes.surfaces import CORTICAL

# Importing a surface from ZIP
zip_surface_importer_model = ZIPSurfaceImporterModel()
zip_surface_importer_model.uploaded = compute_tvb_data_path('surfaceData', 'cortex_16384.zip')
zip_surface_importer_model.surface_type = CORTICAL
zip_surface_importer_model.should_center = False

operation_gid = tvb_client.launch_operation(default_project_gid, ZIPSurfaceImporter, zip_surface_importer_model)
monitor_operation(tvb_client, operation_gid)

# Get the result of surface import
surface_dto = tvb_client.get_operation_results(operation_gid)[0]
surface_gid = surface_dto.gid

surface_gid

Current operation GID aab05f00-7b33-11ea-b3cb-eb2a08972b7e => STATUS 4-PENDING
Operation aab05f00-7b33-11ea-b3cb-eb2a08972b7e has finished with status: 5-FINISHED


'a24bcd2160254d2797201b285e100ba4'

In [7]:
from tvb.adapters.uploaders.region_mapping_importer import RegionMappingImporterModel, RegionMappingImporter

# Importing a region mapping
rm_importer_model = RegionMappingImporterModel()
rm_importer_model.mapping_file = compute_tvb_data_path('regionMapping', 'regionMapping_16k_76.txt')
rm_importer_model.connectivity = connectivity_dto.gid
rm_importer_model.surface = surface_gid

operation_gid = tvb_client.launch_operation(default_project_gid, RegionMappingImporter, rm_importer_model)
monitor_operation(tvb_client, operation_gid)

# Get the result of region mapping import
region_mapping_dto = tvb_client.get_operation_results(operation_gid)[0]
region_mapping_gid = region_mapping_dto.gid

region_mapping_gid

Current operation GID adcffc5e-7b33-11ea-b3cb-eb2a08972b7e => STATUS 4-PENDING
Operation adcffc5e-7b33-11ea-b3cb-eb2a08972b7e has finished with status: 5-FINISHED


'6049015a0f1742c48f4b648df354de6f'

## Load the region mapping together with references information in 3 different ways


In [8]:
# 1.Download and load the region mapping with all its references
region_mapping_complete = tvb_client.load_datatype_with_full_references(region_mapping_gid, os.getcwd())

"This region mapping is linked to a connectivity with GID={} and number_of_regions={}".format(
        region_mapping_complete.connectivity.gid, region_mapping_complete.connectivity.number_of_regions)

'This region mapping is linked to a connectivity with GID=f247b79e-6b1d-449d-a659-9e235494c32e and number_of_regions=96'

In [9]:
# 2.Download and load the region mapping with only GIDs for its references
region_mapping_with_links = tvb_client.load_datatype_with_links(region_mapping_gid, os.getcwd())

"This region mapping is linked to a connectivity with GID={}".format(region_mapping_with_links.connectivity.gid)

'This region mapping is linked to a connectivity with GID=f247b79e-6b1d-449d-a659-9e235494c32e'

In [10]:
# 3.1.Only download the region mapping on client machine
region_mapping_path = tvb_client.retrieve_datatype(region_mapping_dto.gid, os.getcwd())
print(region_mapping_path)

# 3.2.Load the region mapping that was already downloaded on client machine
local_region_mapping_with_links = tvb_client.load_datatype_from_file(region_mapping_path)

"This region mapping is linked to a connectivity with GID={}".format(local_region_mapping_with_links.connectivity.gid)

/home/tvb_user/tvb-root/tvb_documentation/demos/RegionMapping_6049015a0f1742c48f4b648df354de6f.h5


'This region mapping is linked to a connectivity with GID=f247b79e-6b1d-449d-a659-9e235494c32e'

## Launch operation to run a Degree Analyzer over the Connectivity


In [11]:
from tvb.adapters.analyzers.bct_degree_adapters import Degree
from tvb.adapters.analyzers.bct_adapters import BaseBCTModel

# Prepare BCT model
bct_model = BaseBCTModel()
bct_model.connectivity = connectivity_gid

# Launch BCT
operation_gid = tvb_client.launch_operation(default_project_gid, Degree, bct_model)
monitor_operation(tvb_client, operation_gid)

# Get the result of BCT...")
bct_dto = tvb_client.get_operation_results(operation_gid)[0]

bct_dto.gid

Current operation GID b1f4c47c-7b33-11ea-b3cb-eb2a08972b7e => STATUS 4-PENDING
Operation b1f4c47c-7b33-11ea-b3cb-eb2a08972b7e has finished with status: 5-FINISHED


'175d77ccee2f4f28aafa6ad106640604'